In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16361226
paper_name = 'cai_becker_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load data

In [5]:
# The raw data processing for this paper was very complicated. It was originally written in Matlab, so we decided not to re-write it in python.
# The python part only ensures that the ORF labels are cleaned up using python scripts.

In [6]:
original_data = pd.read_csv('raw_data/processed_data.txt',sep='\t')

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4712 x 4


In [8]:
original_data.head()

,ORF,108,245,246
0,BY4742,1.302,1.305,1.017
1,EMPTY,6.362,10.224,6.948
2,YAL002W,0.412,0.686,0.617
3,YAL004W,1.130,2.935,0.978
4,YAL005C,0.446,0.378,0.810


In [9]:
original_data['orf'] = original_data['ORF'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                ORF    108     245    246     orf
index_input                                      
0            BY4742  1.302   1.305  1.017  BY4742
1             EMPTY  6.362  10.224  6.948   EMPTY


In [13]:
original_data = original_data.loc[t,:]

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['108','245','246']].apply(pd.to_numeric, axis=1, errors='coerce')

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4687, 3)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [int(c) for c in original_data.columns.values]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,108,245,246
data_type,value,value,value
orf,,,
YAL002W,0.412,0.686,0.617
YAL004W,1.130,2.935,0.978
YAL005C,0.446,0.378,0.810
YAL007C,1.565,2.668,1.112
YAL008W,1.038,0.802,1.092


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,108,245,246
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,0.412,0.686,0.617
1863,YAL004W,1.130,2.935,0.978
4,YAL005C,0.446,0.378,0.810
5,YAL007C,1.565,2.668,1.112
6,YAL008W,1.038,0.802,1.092


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,108,245,246,108,245,246
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,0.412,0.686,0.617,-1.516632,-0.067112,-2.018376
1863,YAL004W,1.130,2.935,0.978,0.220313,1.352616,-0.354829
4,YAL005C,0.446,0.378,0.810,-1.434381,-0.261544,-1.129000
5,YAL007C,1.565,2.668,1.112,1.272641,1.184067,0.262665
6,YAL008W,1.038,0.802,1.092,-0.002248,0.006115,0.170502


# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 16361226...
Inserting the new data...


100%|██████████| 3/3 [00:22<00:00,  7.34s/it]

Updating the data_modified_on field...
